In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing import image
import torch
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import random

In [2]:
def load_dataset():
    raw = []
    c = 0
    test_image = []
    test_label = []
    train_image = []
    train_label = []
    rootdir = "/content/PlantVillage-Dataset/raw/color"
    for subdir, dirs, files in os.walk(rootdir):
        if subdir != rootdir:
            label = subdir[len(rootdir)+1:]
            #pch = label.split("___")
            #specie_label = pch[0]
            #status_label = pch[1]
            for file in files:
              path = subdir+"/"+ file
              image = Image.open(path)
              transform = transforms.Compose([transforms.PILToTensor()])
              img_tensor = transform(image)
              raw.append((img_tensor,label))
    random.shuffle(raw)
    for item in raw:
      if c<3999:
        test_image.append(item[0])
        test_label.append(item[1])
      else:
        train_image.append(item[0])
        train_label.append(item[1])
      c=c+1
    return (train_image, train_label, test_image, test_label)


In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(40))

In [6]:
rootdir = "/content/PlantVillage-Dataset/raw/color"
ds_train= tf.keras.preprocessing.image_dataset_from_directory(
	rootdir,
	labels='inferred',
	label_mode = "int",
	class_names=['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy'],
	color_mode = 'rgb',
	batch_size = 32,
	image_size = (256,256),
	shuffle = True,
	seed = 123,
	validation_split = 0.1,
	subset = "training",
)

ds_validate= tf.keras.preprocessing.image_dataset_from_directory(
	rootdir,
	labels='inferred',
	label_mode = "int",
	class_names=['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy'],
	color_mode = 'rgb',
	batch_size = 32,
	image_size = (256,256),
	shuffle = True,
	seed = 123,
	validation_split = 0.1,
	subset = "validation",
)

Found 54305 files belonging to 38 classes.
Using 48875 files for training.
Found 54305 files belonging to 38 classes.
Using 5430 files for validation.


In [ ]:
#train_images, test_images = train_images / 255.0, test_images / 255.0
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(ds_train, epochs=10, verbose=2)

Epoch 1/4


In [ ]:
test_loss, test_acc = model.evaluate(ds_validate, verbose=2)
print(test_acc)

In [ ]:
model.save("PlantCare.h5")